# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1o2lpxoi9heyQV1hIlsHXWSfDkBPtze-V?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando preços na internet

In [17]:
from selenium.webdriver.chrome.options import Options 
#chrome_options = Options() -> não funcionou
#chrome_options.headless = True # also works
#nav = webdriver.Chrome(options=chrome_options)

nav = webdriver.Chrome()

#dolar
nav.get('https://www.google.com/')
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar')
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dolar = nav.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
#print(cotacao_dolar)

#euro
nav.get('https://www.google.com/')
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = nav.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
#print(cotacao_euro)

#ouro
nav.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = nav.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',','.')
print(cotacao_ouro)
nav.quit()

281.68


- Importando base de dados

In [8]:
import pandas as pd

produtos_df = pd.read_excel('Produtos.xlsx')
display(produtos_df)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


### Agora vamos exportar a nova base de preços atualizada

In [9]:
produtos_df.loc[produtos_df['Moeda']=='Dólar', 'Cotação'] = float(cotacao_dolar)
produtos_df.loc[produtos_df['Moeda']=='Euro', 'Cotação'] = float(cotacao_euro)
produtos_df.loc[produtos_df['Moeda']=='Ouro', 'Cotação'] = float(cotacao_ouro)
display(produtos_df)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,4.926000,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,5.868098,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,4.926000,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,4.926000,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,5.868098,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,4.926000,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,281.680000,7000.00,1.15,8050.000


In [10]:
#atualizando a tabela
produtos_df['Preço Base Reais'] = produtos_df['Cotação']*produtos_df['Preço Base Original']
produtos_df['Preço Final'] = produtos_df['Margem']*produtos_df['Preço Base Reais']
display(produtos_df)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,4.926000,4925.95074,1.40,6896.331036
1,Carro Renault,4500.00,Euro,5.868098,26406.43875,2.00,52812.877500
2,Notebook Dell,899.99,Dólar,4.926000,4433.35074,1.70,7536.696258
3,IPhone,799.00,Dólar,4.926000,3935.87400,1.70,6690.985800
4,Carro Fiat,3000.00,Euro,5.868098,17604.29250,1.90,33448.155750
5,Celular Xiaomi,480.48,Dólar,4.926000,2366.84448,2.00,4733.688960
6,Joia 20g,20.00,Ouro,281.680000,5633.60000,1.15,6478.640000


In [11]:
#Exportando Valores
produtos_df.to_excel('Produtos Atualizado.xlsx', index = False)